# 리스트 정리 및 신규 파일 추가
- 리스트에 없는 롬을 리스트에 추가한다.
- 리스트에만 있고 실제로는 존재하지 않는 롬을 리스트에서 제거한다.

## 환경변수 설정

In [7]:
# local root
rootdir = 'D:\\Retropie'
dirs = [
    'psx',
       ]
exts = ['.zip', '.chd', '.dsk', '.m3u', '.cdi', '.cue','.fds']
xmlfile = 'gamelist.xml'

## 현재 폴더에서 롬 목록 가져오기

In [8]:
def getRomList(dir):
    import os
    curr = os.path.join(rootdir, dir)
    os.chdir(curr)
    print(os.getcwd())
    allfiles = os.listdir()
    return  [f for f in allfiles if os.path.splitext(f)[1] in exts] 

In [9]:
def trimAndSaveGameList():
    import os
    import xml.etree.ElementTree as ET
    tree = ET.parse(xmlfile)
    games = tree.getroot()
    i = 0
    find = 0
    for game in games:        
        path = game.find('path').text
        if not os.path.isfile(path):
            print("Remove " + path)
            games.remove(game)
            find += 1
        else:
            desc = game.find('desc')
            if desc != None:
                game.remove(desc)
            i += 1
    print("{} game removed".format(find))
    tree.write('gamelist.xml', 'UTF-8')

In [10]:
def findMissingGames(files):
    import os
    # create file dict using game list
    fileDict = {}
    import xml.etree.ElementTree as ET
    tree = ET.parse(xmlfile)
    games = tree.getroot()
    for game in games:
        path = os.path.basename(game.find('path').text)
        fileDict[path] = 1
        
        if game.find('name') == None:
            name = ET.Element('name')
            name.text = game.find('path').text[2:-4]
            game.insert(1, name)  
            print("없는 타이틀 채우기: " + game.find('name').text)
    # check
    missing = []
    for file in files:
        if not file in fileDict:
            missing.append(file)
            print(file + " is not in list")            
    
    tree.write('gamelist.xml', 'UTF-8')
    return missing

In [11]:
def addGameToList(missing):
    
    import os
    import xml.etree.ElementTree as ET
    tree = ET.parse(xmlfile)
    games = tree.getroot()

    sample = '''
	<game>
		<path>PATH</path>
		<name>NAME</name>
		<image>IMAGE</image>
		<releasedate></releasedate>
		<developer></developer>
		<publisher></publisher>
		<genre></genre>
	</game>
'''
    for item in missing:
        print("Add " + item)
        title = item[:-4]
        game = ET.fromstring(sample)
        game.find('path').text = './' + item
        game.find('name').text = title
        game.find('image').text = './box/' + title + '.png'
        for attr in game:
            print(attr.tag, attr.text)
        print("============")
        games.append(game)
    
    tree.write('gamelist.xml', 'UTF-8')

In [12]:
for dir in dirs:
    roms = getRomList(dir)
    trimAndSaveGameList()
    missing = findMissingGames(roms)
    addGameToList(missing)

D:\Retropie\psx
0 game removed
Detana Twinbee Yahho! Deluxe Pack (J).chd is not in list
Super Robot Taisen Alpha (J_K) (v1.1).chd is not in list
Final Fantasy VII International (J-K).m3u is not in list
Add Detana Twinbee Yahho! Deluxe Pack (J).chd
path ./Detana Twinbee Yahho! Deluxe Pack (J).chd
name Detana Twinbee Yahho! Deluxe Pack (J)
image ./box/Detana Twinbee Yahho! Deluxe Pack (J).png
releasedate None
developer None
publisher None
genre None
Add Super Robot Taisen Alpha (J_K) (v1.1).chd
path ./Super Robot Taisen Alpha (J_K) (v1.1).chd
name Super Robot Taisen Alpha (J_K) (v1.1)
image ./box/Super Robot Taisen Alpha (J_K) (v1.1).png
releasedate None
developer None
publisher None
genre None
Add Final Fantasy VII International (J-K).m3u
path ./Final Fantasy VII International (J-K).m3u
name Final Fantasy VII International (J-K)
image ./box/Final Fantasy VII International (J-K).png
releasedate None
developer None
publisher None
genre None
